In [1]:
%load_ext autoreload
%autoreload 2


# Import Libraries

In [2]:
import os

import pandas as pd
from catboost import Pool, CatBoostClassifier

# Read Data

In [3]:
path = "../"
train_data = pd.read_csv(os.path.join(path, "train.csv"))
test_data = pd.read_csv(os.path.join(path, "test.csv"))
print(f"Number of rows and columns in the train data set: {train_data.shape}")
print(f"Number of rows and columns in the test data set: {test_data.shape}")
train_data.head()

Number of rows and columns in the train data set: (48665, 2)
Number of rows and columns in the test data set: (12167, 1)


,rate,text
0,4,Очень понравилось. Были в начале марта с соба...
1,5,В целом магазин устраивает.\nАссортимент позво...
2,5,"Очень хорошо что открылась 5 ка, теперь не над..."
3,3,Пятёрочка громко объявила о том как она заботи...
4,3,"Тесно, вечная сутолока, между рядами трудно ра..."


In [4]:
train_data.groupby("rate").describe()

text                               
      count unique                top freq
rate                                      
1      4138   4130             Грязно    3
2      2410   2407  Отстойный магазин    2
3      6126   6070          Нормально    7
4      9922   9763               Норм   14
5     26069  24804    Хороший магазин  107

# Preparing the data and creating Catboost model

In [5]:
X_train = train_data["text"]
y_train = train_data["rate"]

X_test = test_data["text"]


model = CatBoostClassifier(
    iterations=100,
    depth=5,
    random_seed=42
)

model.fit(
    X_train,
    y_train,
    text_features=[0],
    verbose=True
)

Learning rate set to 0.5
0:	learn: 1.1143203	total: 98.2ms	remaining: 9.72s
1:	learn: 1.0216381	total: 153ms	remaining: 7.52s
2:	learn: 0.9782772	total: 210ms	remaining: 6.78s
3:	learn: 0.9551523	total: 262ms	remaining: 6.28s
4:	learn: 0.9461749	total: 315ms	remaining: 5.98s
5:	learn: 0.9399748	total: 368ms	remaining: 5.76s
6:	learn: 0.9328492	total: 423ms	remaining: 5.62s
7:	learn: 0.9318007	total: 470ms	remaining: 5.41s
8:	learn: 0.9284937	total: 520ms	remaining: 5.26s
9:	learn: 0.9266864	total: 567ms	remaining: 5.1s
10:	learn: 0.9243191	total: 616ms	remaining: 4.98s
11:	learn: 0.9220050	total: 668ms	remaining: 4.89s
12:	learn: 0.9202830	total: 717ms	remaining: 4.8s
13:	learn: 0.9175563	total: 772ms	remaining: 4.74s
14:	learn: 0.9170312	total: 818ms	remaining: 4.64s
15:	learn: 0.9159688	total: 868ms	remaining: 4.55s
16:	learn: 0.9153561	total: 915ms	remaining: 4.46s
17:	learn: 0.9149921	total: 961ms	remaining: 4.38s
18:	learn: 0.9143516	total: 1.01s	remaining: 4.3s
19:	learn: 0.91335

# Predict

In [6]:
# Preparing data in Pool format
dataset_test = Pool(
    data=X_test,
    text_features=[0]
)
predict_classes = model.predict(dataset_test)
predictions = predict_classes

# Create submission

In [7]:
sample_submission = pd.read_csv(os.path.join(path, "sample_submission.csv"))
sample_submission["rate"] = predictions
sample_submission.head()

,rate
0,5
1,5
2,5
3,4
4,5


In [8]:
sample_submission.to_csv("submission.csv", index=False)